# Solving political districting using GA

### Toronot's Neighbourhoods & Communities

Data is available in Appendix B of the book's GitHub repo and can be read directly using URL.

In [14]:
import geopandas as gpd
import pandas as pd
import folium

# Uncomment the following lines if you read the data from a local folder
# toronto = gpd.read_file(r"data/toronto.geojson")
# df = pd.read_csv('data/Toronto_Neighborhoods.csv')

# Read Toronto region adminstration boundries
data_url="https://raw.githubusercontent.com/Optimization-Algorithms-Book/Code-Listings/main/Appendix%20B/data/PoliticalDistricting/"

toronto = gpd.read_file(data_url+"toronto.geojson")
neighborhoods = pd.read_csv(data_url+"Toronto_Neighborhoods.csv")

### Describe and visualize the data

In [15]:
# Pick a sample to represent neighborhoods in Toronto EAST
range_limit = 16 # number of neighborhoods
toronto_sample = toronto.tail(range_limit)
toronto_sample.head(5)

,name,cartodb_id,created_at,updated_at,geometry
124,Bendale,125,2013-02-21 04:41:39.809000+00:00,2013-02-21 04:41:40.047000+00:00,"MULTIPOLYGON (((-79.24460 43.74577, -79.24484 ..."
125,Scarborough Village,126,2013-02-21 04:41:39.809000+00:00,2013-02-21 04:41:40.047000+00:00,"MULTIPOLYGON (((-79.20889 43.74061, -79.20853 ..."
126,Woburn,127,2013-02-21 04:41:39.809000+00:00,2013-02-21 04:41:40.047000+00:00,"MULTIPOLYGON (((-79.20564 43.76405, -79.20492 ..."
127,Guildwood,128,2013-02-21 04:41:39.809000+00:00,2013-02-21 04:41:40.047000+00:00,"MULTIPOLYGON (((-79.17063 43.75566, -79.17073 ..."
128,Morningside,129,2013-02-21 04:41:39.809000+00:00,2013-02-21 04:41:40.047000+00:00,"MULTIPOLYGON (((-79.19334 43.78533, -79.19068 ..."


In [16]:
values = neighborhoods.tail(range_limit)
values = values.join(toronto_sample["cartodb_id"])
display(values)

,name,population,Median_household_income,cartodb_id
124,Bendale,29960,60292,125
125,Scarborough Village,16724,49568,126
126,Woburn,53485,56186,127
127,Guildwood,9917,87538,128
128,Morningside,17455,59963,129
129,West Hill,27392,56051,130
130,Rouge,20823,46664,131
131,Centennial Scarborough,13362,114897,132
132,Highland Creek,12494,102740,133
133,Malvern,43794,64114,134


In [17]:
# Make Malvern center of the map
m = folium.Map(location=[43.8092, -79.2217], zoom_start=12, tiles='cartodbpositron',
               zoom_control=True, scrollWheelZoom=False, dragging=True)

for index, row in toronto_sample.iterrows():
    # Simplify each region's polygon as intricate details are unnecessary
    sim_geo = gpd.GeoSeries(row['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, name=row['name'], style_function=lambda x: {
                           'fillColor': 'black'})
    income = int(neighborhoods[neighborhoods.name == row['name']]['Median_household_income'].values)
    population = int(neighborhoods[neighborhoods.name == row['name']]['population'].values)
    popup_message = "Name:{},\n Population: {},\n Median_household_income:\n{}".format(
        row['name'], population, income)
    folium.Popup(popup_message).add_to(geo_j)
    geo_j.add_to(m)

m

### Data pre-processing

In [18]:
import numpy as np

# Get the population of each neighborhood
def get_population(lst, table):
    return table["population"].iloc[lst].to_numpy()

# Prepare the population dataset
eval = get_population(range(range_limit), values)

# Represent the adjacentcy relationship among every possible pair of neighborhoods in a boolean matrix
def get_neighboors(database):
    result = []
    for i in range(database['name'].size):
        tmp = np.zeros(database['name'].size)
        geo1 = database.iloc[i]
        for j in range(database['name'].size):
            if i != j:
                geo2 = database.iloc[j]
                if geo1["geometry"].intersects(geo2["geometry"]):
                    tmp[j] = 1
        result.append(tmp)
    return np.stack(result)

neighbor = get_neighboors(toronto_sample)
# neighbor

### Create a PoliticalDistricting problem

In [19]:
import autograd.numpy as anp
from pymoo.core.problem import Problem

class PoliticalDistricting(Problem):
    def __init__(self,
                 num_dist, # number of electoral districts in proposal
                 neighbor, # the adjacency matrix
                 populations, # population for each region
                 margin # margin used lower bound and upperbound
                 ):

        self.populations = populations
        self.average = np.mean(populations)
        super().__init__(n_var=len(self.populations), n_obj=1, n_eq_constr=3, xl=0, xu=num_dist-1, vtype=int)

        self.n_var = len(self.populations) 
        self.n_dist = num_dist
        self.margin = margin
        self.neighbor = neighbor
        self.func = self._evaluate
    
    # Extract the neighborhoods that belong to a specific district
    def _gather(self, x, district):
        return np.where(x==district, 1, 0)
    
    # Caculate the upper and lower bounds based on population
    def _get_bounds(self):
        ub = np.ceil(self.average + self.margin) * (len(self.populations)/self.n_dist)
        lb = np.floor(self.average - self.margin) * (len(self.populations)/self.n_dist)
        return ub, lb
    
    # Get the result whether a electoral district is overpopulated or underpopulated
    def _get_result(self, gathered, ub, lb):
        product = gathered * self.populations
        summed_product = np.sum(product, axis=1)
        return np.where((summed_product > ub), 1, 0) + np.where((summed_product < lb), 1, 0)
    
    # Produce the constraint fit for pymoo problems
    def _get_constraint(self, constraint):
        constraint = np.stack(constraint)
        return np.any(constraint==0, axis=0)
    
    # Constraint2: no lone neighborhood within a district unless district only has one neighborhood
    def _get_neighbor(self, gathered):
        singleton = np.sum(gathered, axis=1)
        singleton = np.where(singleton==1, True, False) 
        tmp_neighbor = np.dot(gathered, self.neighbor)
        tmp_neighbor = np.where(tmp_neighbor > 0, 1, 0)
        product = gathered * tmp_neighbor
        return np.all(np.equal(product, gathered), axis=1) + singleton 
   
   # Constraint3: best approximation to make an electoral district a contigouous block
    def cap_district(self, gathered):
        result = np.zeros(gathered.shape[0])
        for i in range(len(gathered)):
            nonzeros = np.nonzero(gathered[i])[0]
            if nonzeros.size != 0:
                mx = np.max(nonzeros)
                mn = np.min(nonzeros)
                result[i] = self.neighbor[mx][mn] or (mx == mn)  
        return result 

    def _evaluate(self, x, out, *args, **kwargs):
        x=np.round(x).astype(int) # Ensure X is binary
        pop_count = []
        constraint1 = []
        constraint2 = []
        constraint3 = []
        for i in range(self.n_dist):
            gathered = self._gather(x, i)
            ub, lb = self._get_bounds()
            result = self._get_result(gathered, ub, lb)
            pop_count.append(result)
            constraint1.append(np.sum(gathered, axis=1)) # make sure that there is no empty district
            constraint2.append((self._get_neighbor(gathered))) # ctrl+f constraint2 for explanation 
            constraint3.append(self.cap_district(gathered)) # ctrl+f constraint3 for explanation 

        holder = np.sum(np.stack(pop_count), axis=0)
        out["F"] = np.expand_dims(holder, axis=1) 
        out["H"] = [self._get_constraint(constraint1), self._get_constraint(constraint2), 
                    self._get_constraint(constraint3)] 

def create_districting_problem(number_of_districts, neighborlist, population_list, margin, seed=1):
    anp.random.seed(seed)
    problem = PoliticalDistricting(number_of_districts, neighborlist, population_list, margin)
    return problem

### Define GA solver and apply it to the problem

In [20]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.pntx import PointCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from pymoo.operators.repair.rounding import RoundingRepair
from pymoo.termination import get_termination
from pymoo.optimize import minimize

num_districts = 3
margin=6000

problem = create_districting_problem(num_districts, neighbor, eval, margin, seed=1)

algorithm = GA(
    pop_size=2000,
    sampling=FloatRandomSampling(),
    crossover=PointCrossover(prob=0.8, n_points=2),
    mutation = PolynomialMutation(prob=0.3, repair=RoundingRepair()),
    eliminate_duplicates=True
)

termination = get_termination("n_gen", 100)

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)        

n_gen  |  n_eval  |     cv_min    |     cv_avg    |     f_avg     |     f_min    
     1 |     2000 |  0.000000E+00 |  1.9858014000 |  3.0000000000 |  3.0000000000
     2 |     4000 |  0.000000E+00 |  1.9298070000 |  3.0000000000 |  3.0000000000
     3 |     6000 |  0.000000E+00 |  1.8748125000 |  3.0000000000 |  3.0000000000
     4 |     8000 |  0.000000E+00 |  1.8098190000 |  2.3750000000 |  2.0000000000
     5 |    10000 |  0.000000E+00 |  1.7318268000 |  2.4166666667 |  2.0000000000
     6 |    12000 |  0.000000E+00 |  1.6098390000 |  2.5294117647 |  2.0000000000
     7 |    14000 |  0.000000E+00 |  1.4323567500 |  2.5172413793 |  2.0000000000
     8 |    16000 |  0.000000E+00 |  1.1648835000 |  2.4150943396 |  1.0000000000
     9 |    18000 |  0.000000E+00 |  0.9574042500 |  2.4588235294 |  1.0000000000
    10 |    20000 |  0.000000E+00 |  0.9349065000 |  2.5076923077 |  1.0000000000
    11 |    22000 |  0.000000E+00 |  0.9089091000 |  2.5000000000 |  0.000000E+00
    12 |    2400

In [21]:
# Print the best solution found
print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

# Print the number of neighborhoods in each district
neighbor_name=np.array(values.name)

sets = {}
for i in range(num_districts):
    sets[i] = []
for i, x in enumerate(res.X):
    sets[x].append(neighbor_name[i])

for i in range(num_districts):
    print("Political District-", i+1, ": ", sets[i])

Best solution found: 
X = [1 1 0 1 1 1 2 1 0 0 2 2 2 1 2 2]
F = [0.]
Political District- 1 :  ['Woburn', 'Highland Creek', 'Malvern']
Political District- 2 :  ['Bendale', 'Scarborough Village', 'Guildwood', 'Morningside', 'West Hill', 'Centennial Scarborough', 'Agincourt South-Malvern West']
Political District- 3 :  ['Rouge', 'Hillcrest Village', 'Steeles', "L'Amoreaux", 'Milliken', 'Agincourt North']


In [22]:
# make Malvern center of the map
m = folium.Map(location=[43.8092, -79.2217], zoom_start=12, tiles='cartodbpositron',
               zoom_control=True, scrollWheelZoom=False, dragging=True)
colors = ["red", "blue", "yellow", "gray", "orange", "green", "cyan", "purple", "pink"] # add more colors as you see fit
import json
sol = res.X
def getcolor(x):
    return colors[int(x["id"])]

for i in range(len(sol)):
    row = toronto_sample.iloc[i]
    sim_geo = gpd.GeoSeries(row['geometry']).simplify(tolerance=0.001)
   
    geo_j = sim_geo.to_json()
    tmp = json.loads(geo_j)
    tmp["features"][0]["id"] = str(sol[i])
    
    result = folium.GeoJson(data=tmp, name=row['name'], style_function=lambda x: {"color":getcolor(x)})
    result.add_to(m)
m